In [78]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.visualize as visualize

notebook_name = "cpol_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2005-11-13T16:00:00"
end = "2005-11-13T18:00:00"

output_directory = base_local / "runs/cpol_demo_geographic"
options_directory = output_directory / "options"

if output_directory.exists():
    shutil.rmtree(output_directory / "options")

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)
# Restrict the ERA5 data to a smaller region containing the CPOL radar
lon_range = [129, 133]
lat_range = [-14, -10]
era5_pl_options = data.era5.data_options(
    start=start, end=end, latitude_range=lat_range, longitude_range=lon_range
)
era5_sl_options = data.era5.data_options(
    start=start,
    end=end,
    data_format="single-levels",
    latitude_range=lat_range,
    longitude_range=lon_range,
)
data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory)

grid_options = grid.create_options(name="geographic")
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol")
option.save_track_options(track_options, options_directory)

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
# visualize_options["middle_echo"] = visualize.option.runtime_options(
#     "middle_echo", save=True, style="presentation", figure_types=["mask"]
# )
# visualize.option.save_display_options(visualize_options, options_directory)
visualize_options = None

2024-09-30 21:17:56,766 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/data.yml
2024-09-30 21:17:56,804 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-09-30 21:17:56,805 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-09-30 21:17:56,806 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/grid.yml
2024-09-30 21:17:56,826 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/track.yml


In [82]:
if output_directory.exists():
    shutil.rmtree(output_directory)
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory
)

2024-09-30 21:19:55,579 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_geographic.
2024-09-30 21:19:55,580 - thor.track - INFO - Beginning simultaneous tracking.
2024-09-30 21:19:55,758 - thor.track - INFO - Processing 2005-11-13T16:00:08.
2024-09-30 21:19:55,763 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T16:00:08.
2024-09-30 21:19:55,765 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.160000.nc
2024-09-30 21:19:56,577 - thor.track - INFO - Processing hierarchy level 0.
2024-09-30 21:19:56,578 - thor.track - INFO - Tracking cell.
2024-09-30 21:19:56,623 - thor.match.match - INFO - Matching cell objects.
2024-09-30 21:19:56,625 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-09-30 21:19:56,632 - thor.visualize.runtime - INFO - Generating runtime visualizations.
2024-09-30 21:19:56,633 - thor.track - INFO - Tracking middle_echo.
2024-09-30 21:19:56,651

In [94]:
import thor.test.test_aura as test_aura

In [95]:
test_aura.test_cpol()

2024-09-30 21:54:29,977 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/data.yml
2024-09-30 21:54:30,008 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-09-30 21:54:30,009 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-09-30 21:54:30,011 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/grid.yml
2024-09-30 21:54:30,029 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/track.yml
2024-09-30 21:54:30,050 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_geographic.
2024-09-30 21:54:30,051 - thor.track - INFO - Beginning simultaneous tracking.
2024-09-30 21:54:30,111 - thor.track - INFO - Processing 2005-11-13T14:00:09.
2024-09-30 21:54:30,112 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T14:00:09.
2024

KeyError: 'track'

# Cartesian Coordinates

In [28]:
output_directory = base_local / "runs/cpol_demo_cartesian"
options_directory = output_directory / "options"

if output_directory.exists():
    shutil.rmtree(output_directory)

grid_options = grid.create_options(name="cartesian", regrid=False)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)
data.option.save_data_options(data_options, options_directory)
option.save_track_options(track_options, options_directory)
# visualize.option.save_display_options(visualize_options, options_directory)


2024-09-27 20:22:15,494 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-09-27 20:22:15,495 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/grid.yml
2024-09-27 20:22:15,500 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/data.yml
2024-09-27 20:22:15,543 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/track.yml


In [29]:
if output_directory.exists():
    shutil.rmtree(output_directory / "attributes")
    shutil.rmtree(output_directory / "masks")
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory,
)

2024-09-27 20:22:16,523 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_cartesian.
2024-09-27 20:22:16,524 - thor.track - INFO - Beginning simultaneous tracking.
2024-09-27 20:22:16,591 - thor.track - INFO - Processing 2005-11-13T16:00:08.
2024-09-27 20:22:16,592 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T16:00:08.
2024-09-27 20:22:16,593 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.160000.nc
2024-09-27 20:22:16,719 - thor.track - INFO - Processing hierarchy level 0.
2024-09-27 20:22:16,721 - thor.track - INFO - Tracking cell.
2024-09-27 20:22:16,757 - thor.match.match - INFO - Matching cell objects.
2024-09-27 20:22:16,759 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-09-27 20:22:16,768 - thor.visualize.runtime - INFO - Generating runtime visualizations.
2024-09-27 20:22:16,769 - thor.track - INFO - Tracking middle_echo.
2024-09-27 20:22:16,787 

# Analysis

In [86]:
import thor.analyze as analyze

In [87]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_directory)
analyze.mcs.quality_control(output_directory, analysis_options)
analyze.mcs.classify_all(output_directory)

2024-09-30 21:46:38,609 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/velocities.csv
2024-09-30 21:46:38,614 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/velocities.yml
2024-09-30 21:46:52,011 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/quality.csv
2024-09-30 21:46:52,019 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/quality.yml
2024-09-30 21:46:52,193 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/classification.csv
2024-09-30 21:46:52,197 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/classification.yml


,,stratiform_offset,inflow,relative_stratiform_offset,tilt,propagation
time,universal_id,,,,,
2005-11-13 06:00:08,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 06:10:23,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 06:20:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 06:30:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 06:40:09,1,left,left,left,shear-perpendicular,shear-perpendicular
...,...,...,...,...,...,...
2005-11-13 20:00:08,4,left,left,leading,up-shear,up-shear
2005-11-13 20:10:23,4,leading,left,leading,up-shear,up-shear
2005-11-13 20:20:08,4,leading,front,leading,up-shear,up-shear


In [17]:
import thor.analyze.utils as utils

In [18]:
options = utils.read_options(output_directory)

In [62]:
import thor.attribute as attribute

In [77]:
csv_filepath = output_directory / "records/filepaths/cpol.csv"
filepaths_df = attribute.utils.read_attribute_csv(csv_filepath)
filepaths_df.iloc[0]["cpol"]
time = filepaths_df.iloc[0].name
filepath = filepaths_df.iloc[0].values[0]
filepath

'/home/ewan/THOR_output/input_data/raw/cpol/cpol_level_1b/v2020/gridded/grid_150km_2500m/2005/20051113/twp10cpolgrid150.b2.20051113.160000.nc'